Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [ ]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [ ]:
import pandas as pd

# Read input files

In [ ]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

In [ ]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [ ]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

In [ ]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

In [ ]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

In [ ]:
candidates_last_purchase = transactions.copy()

In [ ]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])

candidates_last_purchase.week=weeks

### Bestsellers candidates

In [ ]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [ ]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [ ]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [ ]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

In [ ]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [ ]:
transactions.drop_duplicates(['week', 'customer_id'])

In [ ]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [ ]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [ ]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [ ]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [ ]:
transactions['purchased'] = 1

In [ ]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [ ]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [ ]:
data.purchased.mean()

### Add bestseller information

In [ ]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [ ]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [ ]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [ ]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [ ]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [ ]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank',
'buys_womens_clothing', 'buys_mens_clothes', 'buys_kid_baby_clothes', 'season',
'fabric_jeans','fabric_cotton', 'fabric_wool', 'fabric_polyester', 'fabric_silk', 'fabric_denim', 'fabric_linen', 'fabric_spandex', 'fabric_rayon', 'fabric_nylon', 'fabric_leather', 'fabric_suede']
#  'year', 'month',

In [ ]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

# Model training

In [ ]:
from lightgbm.sklearn import LGBMRanker

In [ ]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [ ]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

In [ ]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

# Calculate predictions

In [ ]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

# Create submission

Alterd = only repurchase information

In [ ]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
subAltered = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
# Create a dictionary of owned articles for each customer
owned_articles = transactions.groupby('customer_id')['article_id'].apply(set).to_dict()

# Define a function to replace owned articles with '000000000'
def replace_owned_articles(customer_id, predictions, owned_articles):
    new_predictions = []
    for article_id in predictions:
        if article_id in owned_articles.get(customer_id, set()):
            new_predictions.append(article_id)
        else:
            new_predictions.append('000000000')
    return new_predictions


In [ ]:
%%time

preds = []
predsAlterd = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
        
    # Add bestsellers from the previous week
    pred = pred + bestsellers_last_week
        
    # Replace owned articles with '000000000'
    predAltered = replace_owned_articles(c_id, pred, owned_articles)
    
    # Take the top 12 predictions (excluding any replaced '000000000' articles)
    preds.append(pred[:12])
    predsAlterd.append(predAltered[:12])


In [ ]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [ ]:
sub_name = 'RepurchaseFilter_model_submissionUnAltered2'
sub.to_csv(f'{sub_name}.csv.gz', index=False)

In [ ]:
predsAlterd = [' '.join(['0' + str(p) for p in ps]) for ps in predsAlterd]
subAltered.prediction = predsAlterd

In [ ]:
sub_nameAltered = 'RepurchaseFilter_model_submissionAltered2'
subAltered.to_csv(f'{sub_nameAltered}.csv.gz', index=False)

All repurchase information


In [ ]:
# Create a dictionary of owned articles for each customer
owned_articles = transactions.groupby('customer_id')['article_id'].apply(list).to_dict()

def get_latest_bought_articles2(customer_id, owned_articles):
    new_predictions = []
    bought_articles = owned_articles.get(customer_id, [])
    
    # Create a DataFrame with the bought articles for the specific customer
    customer_transactions = transactions[(transactions['customer_id'] == customer_id) & (transactions['article_id'].isin(bought_articles))]
    
    # Sort by purchase date in descending order
    customer_transactions = customer_transactions.sort_values(by='t_dat', ascending=False)
    
    # Take the latest 12 unique bought articles
    latest_bought = customer_transactions['article_id'].unique()[:12]
    
    for i in range(12):
        if i<len(latest_bought):
            new_predictions.append(latest_bought[i])
        else:
            new_predictions.append('000000000')
    return new_predictions



In [ ]:
subLatest = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
%%time
latest_bought_articles = []
for c_id in customer_hex_id_to_int(subLatest.customer_id):
    
    latest_bought = get_latest_bought_articles2(c_id, owned_articles)
    latest_bought_articles.append(latest_bought[:12])


In [ ]:
print(latest_bought_articles[:3])

In [ ]:
df = pd.DataFrame(latest_bought_articles)
df.to_csv('filename.csv', index=False)

In [ ]:
latest_bought_articlesdone = [' '.join(['0' + str(p) for p in ps]) for ps in latest_bought_articles]
subLatest.prediction = latest_bought_articlesdone


In [ ]:
sub_nameLatest = 'model_submission_LatestBought'
subLatest.to_csv(f'{sub_nameLatest}.csv.gz', index=False)